In [ ]:
pip install crewai crewai-tools

In [ ]:
pip install langtrace-python-sdk

In [1]:
# Must precede any llm module imports

from langtrace_python_sdk import langtrace
import os

langtrace.init(api_key = os.getenv("LANGTRACE_API_KEY"))

Initializing Langtrace SDK..
⭐ Leave our github a star to stay on top of our updates - https://github.com/Scale3-Labs/langtrace
Skipping litellm due to error while instrumenting: module 'litellm.main' has no attribute 'image_generation'
Exporting spans to GenAI..
Langtrace Project URL: https://app.langtrace.ai/project/cmhhdkz730001xuyx3wwrx5bc/traces


c:\Leela\genai_workspace\crewenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from crewai import Agent, Task, Crew
from textwrap import dedent

# Here backstory is list of capabilities that Agent can perform
# goal is high level overview of Agent
fin_advisor_agent = Agent(
    role="Financial Advisor",
    goal="Analyze customer financial situation and provide insights",
    backstory=dedent("""
        You are an experienced financial advisor in India with expertise in
         personal finanace management. You help clients understand their financial
         health and provide actionable insights.
    """)
)

In [4]:
fin_analysis_task = Task(
    agent=fin_advisor_agent,
    description=dedent("""
        Analyze the customer's financial situation based on the following inputs:
        - Monthly salary: {salary}
        - Monthly expenses: {expenses}
        - Current savings: {savings}
        - Current investments: {investments}
        - Outstanding loans: {loans}
        - Number of dependents: {dependents}
        
        Provide a detailed analysis including:
        1. Current financial health score (Good/Average/Needs Improvement)
        2. Monthly savings potential
        3. Debt-to-income ratio
        4. Basic recommendations for improvement
        
        Format the output as a structured report with clear sections
    """),
    expected_output=dedent("""
        A detailed financial analysis report with the following sections in markdown format without '```'
         - Financial Health Overview
         - Income and Expense Analysis
         - Savings Analysis
         - Debt Analysis
         - Key Recommendations
    """),
    output_file="analysis_task.md"
)

In [5]:
from crewai.memory.long_term.long_term_memory import LongTermMemory
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai.memory.storage.rag_storage import RAGStorage

fin_advisory_crew = Crew(
    agents=[fin_advisor_agent],
    tasks=[fin_analysis_task],
    verbose=True,
    memory=True,
    long_term_memory=LongTermMemory(
        storage=LTMSQLiteStorage(db_path="my_data_dir/crew/long_term_memory_storage.db")
    ),
    short_term_memory=ShortTermMemory(
        storage=RAGStorage(type="short_term_memory", path="my_data_dir/crew/short_term_memory_storage")
    )
)

In [6]:
inputs = {
    "salary": "₹1,50,000",
    "expenses": "₹80,000",
    "savings": "₹5,00,000",
    "investments": "₹10,00,000",
    "loans": "₹20,00,000",
    "dependents": "2"
}

In [7]:
result = fin_advisory_crew.kickoff(inputs=inputs)

result

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e1d89def-e653-4780-89a5-7a2cd3136229                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Financial Advisor                                                                                       │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Analyze the customer's financial situation based on the following inputs:                                      │
│  - Monthly salary: ₹1,50,000                                                                                    │
│  - Monthly expenses: ₹80,000                                                                                    │
│  - Current savings: ₹5,00,000                                                                                   │
│  - Current investments: ₹10,00,000                                                                              │
│  - Outstanding loans: ₹20,00,000                                                                                │
│  - Number of dependents: 2                                                                                      │
│                                                                                                                 │
│  Provide a detailed analysis including:                                                                         │
│  1. Current financial health score (Good/Average/Needs Improvement)                                             │
│  2. Monthly savings potential                                                                                   │
│  3. Debt-to-income ratio                                                                                        │
│  4. Basic recommendations for improvement                                                                       │
│                                                                                                                 │
│  Format the output as a structured report with clear sections                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Financial Advisor                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Financial Health Overview                                                                                    │
│  Based on the provided financial data, the overall financial health score can be categorized as **Average**.    │
│  While the monthly salary exceeds expenses significantly, the outstanding loans pose a risk that needs to be    │
│  addressed.                                                                                                     │
│                                                                                                                 │
│  # Income and Expense Analysis                                                                                  │
│  - **Monthly Salary:** ₹1,50,000                                                                                │
│  - **Monthly Expenses:** ₹80,000                                                                                │
│                                                                                                                 │
│  **Net Monthly Surplus:** ₹1,50,000 - ₹80,000 = ₹70,000                                                         │
│                                                                                                                 │
│  This positive cash flow indicates that you are currently managing your monthly expenses well relative to your  │
│  salary.                                                                                                        │
│                                                                                                                 │
│  # Savings Analysis                                                                                             │
│  - **Current Savings:** ₹5,00,000                                                                               │
│  - **Monthly Savings Potential:**                                                                               │
│                                                                                                                 │
│  With a monthly surplus of ₹70,000 from your income, if you allocate a portion toward savings:                  │
│  - Potential Monthly Savings: ₹70,000 (assuming you can save all surplus)                                       │
│  - Annual Savings Potential: ₹70,000 x 12 = ₹8,40,000                                                           │
│                                                                                                                 │
│  This would substantially enhance your savings over the coming years.                                           │
│                                                                                                                 │
│  # Debt Analysis                                                                                                │
│  - **Outstanding Loans:** ₹20,00,000                                                                            │
│                                                                                                                 │
│  **Debt-to-Income Ratio (DTI):**                                                                                │
│                                                                                                                 │
│  DTI = (Total Monthly Debt Payments / Monthly Income) x

: 